In [1]:
# imports
import requests
import pandas as pd
import os

In [2]:
yelp_api_key = os.environ["yelp_api_key"]
fsq_api_key = os.environ["fsq_api_key"]

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [3]:
# Define the headers for the Foursquare API call
headers = {
    "accept": "application/json",
    "Authorization": fsq_api_key
}

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [4]:
# Function to fetch Foursquare data for a given latitude and longitude
def fetch_foursquare_data(lat, lon):
    url = f"https://api.foursquare.com/v3/places/search?query=%20&ll={lat}%2C{lon}&radius=1000&categories=13000&sort=RELEVANCE&limit=5"
    response = requests.get(url, headers=headers)
    print(response.json())
    results = response.json().get("results", [])
    
    extracted_data = []
    for result in results:
        name = result["name"]
        distance = result["distance"]
        formatted_address = result["location"]["formatted_address"]
        
        extracted_data.append([lat, lon, name, distance, formatted_address,])
    
    return extracted_data

# File path for the CSV file containing the latitudes and longitudes
file_path = r"C:\Users\wamm1\Desktop\P3-Statistical-Modelling-Project\data\sample_bikes.csv"
df_bikes = pd.read_csv(file_path)

# Fetch Foursquare data for each set of latitude and longitude
all_foursquare_data = []
for _, row in df_bikes.iterrows():
    all_foursquare_data.extend(fetch_foursquare_data(row["bike_latitude"], row["bike_longitude"]))

{'results': [{'fsq_id': '58d4650e2f91cb5ccec03447', 'categories': [{'id': 13029, 'name': 'Brewery', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/brewery_', 'suffix': '.png'}}, {'id': 13065, 'name': 'Restaurant', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_', 'suffix': '.png'}}], 'chains': [], 'distance': 194, 'geocodes': {'drop_off': {'latitude': 43.663731, 'longitude': -79.319985}, 'main': {'latitude': 43.663372, 'longitude': -79.319902}, 'roof': {'latitude': 43.663372, 'longitude': -79.319902}}, 'link': '/v3/places/58d4650e2f91cb5ccec03447', 'location': {'address': '1001 Eastern Ave', 'country': 'CA', 'cross_street': 'Woodfield Rd', 'formatted_address': '1001 Eastern Ave (Woodfield Rd), Toronto ON M4L 1A8', 'locality': 'Toronto', 'postcode': 'M4L 1A8', 'region': 'ON'}, 'name': 'Rorschach Brewing', 'related_places': {}, 'timezone': 'America/Toronto'}, {'fsq_id': '4ad8f034f964a520301621e3', 'categories': [{'id': 13099, 'name': 'Chinese Res

Put your parsed results into a DataFrame

In [5]:
df_all_foursquare = pd.DataFrame(all_foursquare_data, columns=["bike_latitude", "bike_longitude", "name", "distance", "address"])

# Display or save/export the dataframe as needed
print(df_all_foursquare)
df_all_foursquare.to_csv("C:\\Users\\wamm1\\Desktop\\P3-Statistical-Modelling-Project\\data\\df_all_foursquare.csv", index=False)



    bike_latitude  bike_longitude                               name  \
0       43.665269      -79.319796                  Rorschach Brewing   
1       43.665269      -79.319796                        Chino Locos   
2       43.665269      -79.319796                  Hasting Snack Bar   
3       43.665269      -79.319796                  Black Lab Brewing   
4       43.665269      -79.319796                       Udupi Palace   
5       43.671420      -79.445947               The Greater Good Bar   
6       43.671420      -79.445947                    Tre Mari Bakery   
7       43.671420      -79.445947                        Hale Coffee   
8       43.671420      -79.445947                           Parallel   
9       43.671420      -79.445947             Blood Brothers Brewing   
10      43.666224      -79.317693                  Rorschach Brewing   
11      43.666224      -79.317693                The Burger's Priest   
12      43.666224      -79.317693                       Udupi Pa

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [6]:
# Define the headers for the Yelp API call
#headers = {
    #"accept": "application/json",
    #"Authorization": yelp_api_key
    #}
yelp_api_key = os.environ["yelp_api_key"]
headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {yelp_api_key}"
}

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [7]:
# Function to fetch Yelp data for a given latitude and longitude
def fetch_yelp_data(lat, lon):
    url = f"https://api.yelp.com/v3/businesses/search?latitude={lat}&longitude={lon}&term=food&radius=1000&sort_by=review_count&limit=5"
    response = requests.get(url, headers=headers)
    print(response.json())
    businesses = response.json().get("businesses", [])
    
    extracted_data = []
    for business in businesses:
        name = business["name"]
        review_count = business["review_count"]
        categories = ', '.join([category["title"] for category in business["categories"]])
        rating = business["rating"]
        display_address = ', '.join(business["location"]["display_address"])
        display_phone = business["display_phone"]
        distance = business["distance"]

        extracted_data.append([lat, lon, name, review_count, categories, rating, display_address, display_phone, distance])
    
    return extracted_data

# Read the CSV file containing the latitudes and longitudes
file_path = r"C:\Users\wamm1\Desktop\P3-Statistical-Modelling-Project\data\sample_bikes.csv"
df_yelp = pd.read_csv(file_path)


# Fetch Yelp data for each set of latitude and longitude
all_data = []
for _, row in df_yelp.iterrows():
    all_data.extend(fetch_yelp_data(row["bike_latitude"], row["bike_longitude"]))

{'businesses': [{'id': 'dAmVTQ6ukuLh4UxTmBoRkg', 'alias': 'mahas-toronto-2', 'name': "Maha's", 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/Sx-jTXmJ5szMlqR6E9e6dA/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/mahas-toronto-2?adjust_creative=bgFAiaZNQ_2T2qA0ygnbbw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=bgFAiaZNQ_2T2qA0ygnbbw', 'review_count': 598, 'categories': [{'alias': 'egyptian', 'title': 'Egyptian'}, {'alias': 'vegan', 'title': 'Vegan'}, {'alias': 'sandwiches', 'title': 'Sandwiches'}], 'rating': 4.0, 'coordinates': {'latitude': 43.67167, 'longitude': -79.32853}, 'transactions': [], 'price': '$$', 'location': {'address1': '226 Greenwood Avenue', 'address2': '', 'address3': '', 'city': 'Toronto', 'zip_code': 'M4L 2R2', 'country': 'CA', 'state': 'ON', 'display_address': ['226 Greenwood Avenue', 'Toronto, ON M4L 2R2', 'Canada']}, 'phone': '+14164622703', 'display_phone': '+1 416-462-2703', 'distance': 1005.190034713692}, {'id': 'RU

Put your parsed results into a DataFrame

In [8]:
df_all_yelp = pd.DataFrame(all_data, columns=["bike_latitude", "bike_longitude", "name", "review_count", "categories", "rating", "address", "phone", "distance"])

print(df_all_yelp)
df_all_yelp.to_csv("C:\\Users\\wamm1\\Desktop\\P3-Statistical-Modelling-Project\\data\\df_all_yelp.csv", index=False)

    bike_latitude  bike_longitude                                      name  \
0       43.665269      -79.319796                                    Maha's   
1       43.665269      -79.319796                        Lahore Tikka House   
2       43.665269      -79.319796                       The Burger's Priest   
3       43.665269      -79.319796  Gio Rana's Really Really Nice Restaurant   
4       43.665269      -79.319796                              Udupi Palace   
5       43.671420      -79.445947                                      Agio   
6       43.671420      -79.445947                     Khmer Thai Restaurant   
7       43.671420      -79.445947                                  Parallel   
8       43.671420      -79.445947                       Marcello’s pizzeria   
9       43.671420      -79.445947                          Sushi California   
10      43.666224      -79.317693                                    Maha's   
11      43.666224      -79.317693                   

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Comparing results:
    *Foursquare provides multiple geocodes (drop_off, main, roof), which could be useful for different use cases. It also provides an icon URL for the category
    *Yelp provides a direct URL to the business's Yelp page, an image URL for the business, a review count, and a rating. 
    *when it comes to the address, foursqure dives a bit deeper and gives information such as: country, locaility, postal code and region
    *Yelp provides a phone number for the business.
    *Yelp also provides the total business found in the area even though the returned api call was limited to 5. foursqare doesnt tell us how many businesses there are on top of the requested 5
Get the top 10 restaurants according to their rating:   
    *Both api requested where sorted by rating. I divided the top 10 restusrants into 5 each for each API request
'''

Get the top 10 restaurants according to their rating

In [9]:
#can be answered direcrly through the yelp api because it gives ratings.
#url = f"https://api.yelp.com/v3/businesses/search?latitude={lat}&longitude={lon}&term=food&radius=1000&sort_by=review_count&limit=5"
#in the line above of the api call, change sort_by from review_count to rating and limit from 5 to 10.